In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from config import DB_USERNAME, DB_PASSWORD, DB_NAME, DB_PORT_POSTGRESQL, DB_HOST

In [2]:
df = pd.read_csv('universities-clean.csv')

In [3]:
df.head()

,Rank,Institution,City,Country,Continent,Founded,Affiliation,AttendanceMode,Enrollment,Link
0,1,Indira Gandhi National Open University,New Delhi,India,Asia,1985,Public,Distance/In-Person,7140000,https://en.wikipedia.org/wiki/Indira_Gandhi_Na...
1,2,"National University, Bangladesh",Gazipur,Bangladesh,Asia,1992,Public,In-Person,3425832,https://en.wikipedia.org/wiki/National_Univers...
2,3,Anadolu University,Eskişehir,Turkey,Asia,1958,Public,Distance/In-Person,1974343,https://en.wikipedia.org/wiki/Anadolu_University
3,4,California Community Colleges,California,United States,North America,1967,Public,In-Person,1800000,https://en.wikipedia.org/wiki/California_Commu...
4,5,Islamic Azad University,Iran,Iran,Asia,1982,Private,In-Person,1778000,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...


In [5]:
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT_POSTGRESQL,
    password=DB_PASSWORD,
    user=DB_USERNAME,
    database=DB_NAME
)

In [6]:
cursor = conn.cursor()

In [7]:
create_query_file = open('./sql/postgres_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university (\n    id SERIAL PRIMARY KEY,\n    rank BIGINT NOT NULL,\n    name VARCHAR(255) NOT NULL,\n    city VARCHAR(255) NOT NULL,\n    country VARCHAR(255) NOT NULL,\n    continent VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    affiliation VARCHAR(255) NOT NULL,\n    attendancemode VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n    CONSTRAINT unique_combination UNIQUE (country, name)\n);'

In [8]:
cursor.execute(create_query)

DuplicateTable: relation "university" already exists


In [9]:
conn.commit()

In [11]:
conn.rollback()

In [12]:
engine = create_engine("postgresql+psycopg2://", creator=lambda: conn)

In [13]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,rank,name,city,country,continent,founded,affiliation,attendancemode,enrollment,link
0,1,1,Indira Gandhi National Open University,New Delhi,India,Asia,1985,Public,Distance/In-Person,7140000,https://en.wikipedia.org/wiki/Indira_Gandhi_Na...
1,2,2,"National University, Bangladesh",Gazipur,Bangladesh,Asia,1992,Public,In-Person,3425832,https://en.wikipedia.org/wiki/National_Univers...
2,3,3,Anadolu University,Eskişehir,Turkey,Asia,1958,Public,Distance/In-Person,1974343,https://en.wikipedia.org/wiki/Anadolu_University
3,4,4,California Community Colleges,California,United States,North America,1967,Public,In-Person,1800000,https://en.wikipedia.org/wiki/California_Commu...
4,5,5,Islamic Azad University,Iran,Iran,Asia,1982,Private,In-Person,1778000,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...
...,...,...,...,...,...,...,...,...,...,...,...
85,86,85,Aix-Marseille University,Marseille,France,Europe,1409,Public,In-Person,80000,https://en.wikipedia.org/wiki/Aix-Marseille_Un...
86,87,86,University of Hagen,Hagen,Germany,Europe,1974,Public,Distance,69982,https://en.wikipedia.org/wiki/University_of_Hagen
87,88,87,Université de Montréal,Canada,Canada,North America,1878,Public,Distance/In-Person,67109,https://en.wikipedia.org/wiki/Universit%C3%A9_...
88,89,88,University of the Philippines,Philippines,Philippines,Asia,1908,Public,Distance/In-Person,64144,https://en.wikipedia.org/wiki/University_of_th...


In [14]:
data = list(df.itertuples(name=None, index=None))
merge_query_file = open('./sql/postgres_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'INSERT INTO university (\n    rank, name, city, country, continent, founded, \n    affiliation, attendancemode, enrollment, link)\nVALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)\nON CONFLICT (country, name) DO UPDATE\nSET\n    rank=EXCLUDED.rank,\n    name=EXCLUDED.name,\n    city=EXCLUDED.city,\n    country=EXCLUDED.country,\n    continent=EXCLUDED.continent,\n    founded=EXCLUDED.founded,\n    affiliation=EXCLUDED.affiliation,\n    attendancemode=EXCLUDED.attendancemode,\n    enrollment=EXCLUDED.enrollment,\n    link=EXCLUDED.link;'

In [15]:
cursor.executemany(merge_query, data)

In [16]:
conn.commit()

In [41]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,rank,name,city,country,continent,founded,affiliation,attendancemode,enrollment,link
0,1,1,Indira Gandhi National Open University,New Delhi,India,Asia,1985,Public,Distance/In-Person,7140000,https://en.wikipedia.org/wiki/Indira_Gandhi_Na...
1,2,2,"National University, Bangladesh",Gazipur,Bangladesh,Asia,1992,Public,In-Person,3425832,https://en.wikipedia.org/wiki/National_Univers...
2,3,3,Anadolu University,Eskişehir,Turkey,Asia,1958,Public,Distance/In-Person,1974343,https://en.wikipedia.org/wiki/Anadolu_University
3,4,4,California Community Colleges,California,United States,North America,1967,Public,In-Person,1800000,https://en.wikipedia.org/wiki/California_Commu...
4,5,5,Islamic Azad University,Iran,Iran,Asia,1982,Private,In-Person,1778000,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...
...,...,...,...,...,...,...,...,...,...,...,...
85,86,85,Aix-Marseille University,Marseille,France,Europe,1409,Public,In-Person,80000,https://en.wikipedia.org/wiki/Aix-Marseille_Un...
86,87,86,University of Hagen,Hagen,Germany,Europe,1974,Public,Distance,69982,https://en.wikipedia.org/wiki/University_of_Hagen
87,88,87,Université de Montréal,Canada,Canada,North America,1878,Public,Distance/In-Person,67109,https://en.wikipedia.org/wiki/Universit%C3%A9_...
88,89,88,University of the Philippines,Philippines,Philippines,Asia,1908,Public,Distance/In-Person,64144,https://en.wikipedia.org/wiki/University_of_th...
